## Set Up Dependencies and Data


In [ ]:
import joblib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats as scipy_stats
import seaborn as sns
from teeplot import teeplot as tp


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/chaynes2019/AvidaGeneDupe/refs/heads/main/experiments/2022-3-29-PaperDuplication/TidiedData/FinalDominantGenomeLengths.csv")


## Reproducibility


In [ ]:
%load_ext watermark
%watermark -iwbmuvg -iv


In [ ]:
joblib.hash(df)


# Describe data


In [ ]:
df.head()


In [ ]:
df.describe()


In [ ]:
df.nunique()


In [ ]:
df["Treatment"].unique()


In [ ]:
df["Genome Length"] = df["Task Count"]


## Plot Data


In [ ]:
signif = ""
env = "static"


In [ ]:
def order(signif: str="") -> list:
    return [
        "Baseline",
        "High mutation rate",
        "Slip-scatter",
        "Slip-NOP",
        "Slip-random",
        f"Slip-scramble{signif}",
        f"Slip-duplicate{signif}",
    ]


In [ ]:
df_ = df.groupby(
    ["Treatment", "Replicate #"],
).max("Time").reset_index()
df_["Treatment"] = df_[
    "Treatment"
].replace(
    "Baseline-Treatment", "Baseline"
).replace(
    "High-Mutation", "High mutation rate"
).replace(
    "Slip-scramble", f"Slip-scramble{signif}"
).replace(
    "Slip-duplicate", f"Slip-duplicate{signif}",
)

with tp.teed(
    sns.catplot,
    data=df_,
    x="Treatment",
    y="Genome Length",
    hue="Treatment",
    order=order(signif),
    hue_order=order(signif),
    aspect=1.4,
    height=2.5,
    inner="box",
    kind="violin",
    linewidth=1,
    cut=0,
    palette="Set2",
    teeplot_outattrs={"env": env},
) as g:
    ax, = g.axes.flat
    ax.set_ylabel("Genome Length")
    ax.set_xlabel("")
    plt.xticks(rotation=30, ha="right")


## Slice and Dice


In [ ]:
df_ = df.groupby(
    ["Treatment", "Replicate #"],
).max("Time").reset_index()
df_["Treatment"] = df_[
    "Treatment"
].replace(
    "Baseline-Treatment", "Baseline"
).replace(
    "High-Mutation", "High mutation rate"
).replace(
    "Slip-scramble", f"Slip-scramble{signif}"
).replace(
    "Slip-duplicate", f"Slip-duplicate{signif}",
)


In [ ]:
def order2(signif):
    return [
        "Baseline",
        f"Slip-duplicate{signif}",
        "High mutation rate",
        "Slip-scatter",
        "Slip-NOP",
        "Slip-random",
        f"Slip-scramble{signif}",
    ]


In [ ]:
with tp.teed(
    sns.catplot,
    data=df_,
    x="Treatment",
    y="Genome Length",
    hue="Treatment",
    order=order2(signif),
    hue_order=order2(signif),
    aspect=1.4,
    height=2.5,
    cut=0,
    inner="box",
    kind="violin",
    linewidth=1,
    palette="Set2",
    teeplot_outattrs={"env": env},
) as g:
    ax, = g.axes.flat
    # Core lines for your request:
    ax.xaxis.set_ticks_position("bottom")
    ax.xaxis.set_label_position("top")
    ax.tick_params(axis="x", labelbottom=False, labeltop=True)
    ax.set_ylabel("Genome Length")
    ax.set_xlabel("")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="left")


In [ ]:
df_["split"] = df_["Treatment"].map(
    {
        "Baseline": 0,
        "High mutation rate": 1,
        f"Slip-duplicate{signif}": 0,
        f"Slip-scramble{signif}": 1,
        "Slip-NOP": 1,
        "Slip-random": 1,
        "Slip-scatter": 1,
    },
)

with tp.teed(
    sns.catplot,
    data=df_,
    x="Treatment",
    y="Genome Length",
    hue="Treatment",
    col="split",
    order=order2(signif),
    hue_order=order2(signif),
    aspect=0.8,
    height=2.5,
    inner="box",
    kind="violin",
    cut=0,
    sharex=False,
    linewidth=1,
    palette="Set2",
    teeplot_outattrs={"env": env},
) as g:
    g.set(ylabel="Genome Length")
    g.set_titles("")
    for ax in g.axes.flat:
        ax.xaxis.set_ticks_position("bottom")
        ax.xaxis.set_label_position("top")
        ax.tick_params(axis="x", labelbottom=False, labeltop=True)
        ax.set_xlabel("")
        ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="left")


    g.sharex = False
    g.axes.flat[0].set_xlim(None, 1.5)
    g.axes.flat[1].set_xlim(1.5, None)

    gs = g.axes[0, 0].get_gridspec()
    gs.set_width_ratios([2, 5])  # [left, right]

    g.fig.subplots_adjust(wspace=0.2)


## Statistical Tests


In [ ]:
# --------------------------------------------------------------------
# Kruskal-Wallis across all treatment groups
# --------------------------------------------------------------------
grouped = df.groupby("Treatment")["Task Count"].apply(list)
kw_stat, kw_p = scipy_stats.kruskal(*grouped)

print(f"Kruskal-Wallis: H = {kw_stat:.3f}, p = {kw_p:.3e}")

# --------------------------------------------------------------------
# Pair-wise Mann-Whitney U tests
#    - Slip-scramble  vs Slip-NOP
#    - Slip-duplicate vs Slip-NOP
#    - Slip-duplicate vs Slip-scramble
#    Bonferroni-correct p-values for 3 comparisons
# --------------------------------------------------------------------
comparisons = [
    ("Slip-scramble",  "Slip-NOP"),
    ("Slip-duplicate", "Slip-NOP"),
    ("Slip-duplicate", "Slip-scramble")
]

results   = []
raw_ps    = []

for a, b in comparisons:
    x = df.loc[df["Treatment"] == a, "Genome Length"]
    y = df.loc[df["Treatment"] == b, "Genome Length"]

    u, p = scipy_stats.mannwhitneyu(x, y, alternative="two-sided")
    raw_ps.append(p)
    results.append((a, b, u, p))

# Bonferroni adjustment (alpha_nominal * number_of_tests)
bonf_ps = [min(p * len(comparisons), 1.0) for p in raw_ps]

# --------------------------------------------------------------------
# Report
# --------------------------------------------------------------------
for (a, b, u, p_raw), p_adj in zip(results, bonf_ps):
    print(f"Mann-Whitney {a} vs {b}: "
          f"U = {u:.0f}, raw p = {p_raw:.3e}, Bonferroni-corrected p = {p_adj:.3e}")


In [ ]:
df.groupby("Treatment")["Genome Length"].agg(["mean", "std"]).round(2)
